### Modelo XgBoost Mejores Características, Hiper, Normalizados y con Resample

In [61]:
#!pip install boruta
##Importar librerías
import warnings
warnings.filterwarnings('ignore')

##Librerías importantes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import xgboost as xgb
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier


### Paso 1: Revisar Mejores Características

In [9]:
##IMportar la data
X_test_XG = pd.read_csv('./X_test_norm_RES.csv')
X_train_XG = pd.read_csv('./X_train_norm_RES.csv')

y_train_XG =pd.read_csv('./y_train_res_over.csv')
y_test_XG =pd.read_csv('./y_res_test_over.csv')

In [36]:
X_train, y_train = X_train_XG, y_train_XG

# Definir el modelo de XGBoost
model = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='mlogloss')

# Ajustar el modelo
model.fit(X_train, y_train)

# Obtener las importancias de las características
importances = model.feature_importances_

# Crear un DataFrame con las características y sus importancias
feature_importances = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})

# Ordenar el DataFrame por importancia descendente
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Seleccionar las características importantes usando un umbral
selector = SelectFromModel(model, threshold='mean', prefit=True)
X_train_selected = selector.transform(X_train)

# Obtener los índices de las características seleccionadas
selected_indices = selector.get_support(indices=True)

# Obtener los nombres de las características seleccionadas
selected_feature_names = X_train.columns[selected_indices]

# Mostrar las características seleccionadas y sus importancias
selected_feature_importances = feature_importances[feature_importances['Feature'].isin(selected_feature_names)]
print("Características seleccionadas y sus importancias:")
print(selected_feature_importances)



Características seleccionadas y sus importancias:
          Feature  Importance
19        Pedag11    0.157845
4      DemoPadres    0.148152
11         Pedag3    0.076236
7    DemoEsFisico    0.074027
24        Pedag16    0.068848
5   DemoLaborando    0.067497
23        Pedag15    0.062224
1        DemoEdad    0.053598
14         Pedag6    0.041642


### Paso 2: Filtrar X_train y X_test con las mejores características

In [38]:
X_train_fin = X_train_XG[selected_feature_names]

X_test_fin = X_test_XG[selected_feature_names]

In [57]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Definir el grid de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Configurar la búsqueda de hiperparámetros
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Entrenar el modelo
grid_search.fit(X_train_fin, y_train_XG)

# Mejor modelo
best_model = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred = best_model.predict(X_test_fin)

# Evaluar el modelo
accuracy = accuracy_score(y_test_XG, y_pred)
print(f'Best parameters: {grid_search.best_params_}')
print(f'Accuracy: {accuracy}')

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsampl

In [58]:
print(classification_report(y_test_XG, y_pred))

              precision    recall  f1-score   support

           0       0.34      0.87      0.49        15
           1       0.83      0.33      0.48        15
           2       0.00      0.00      0.00        15

    accuracy                           0.40        45
   macro avg       0.39      0.40      0.32        45
weighted avg       0.39      0.40      0.32        45



In [59]:
# Crear y entrenar el modelo
modelDos = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
modelDos.fit(X_train_fin, y_train_XG)

# Predecir en el conjunto de prueba
y_pred = modelDos.predict(X_test_fin)

# Evaluar el modelo
accuracy = accuracy_score(y_test_XG, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.4


In [60]:
print(classification_report(y_test_XG, y_pred))

              precision    recall  f1-score   support

           0       0.34      0.87      0.49        15
           1       0.83      0.33      0.48        15
           2       0.00      0.00      0.00        15

    accuracy                           0.40        45
   macro avg       0.39      0.40      0.32        45
weighted avg       0.39      0.40      0.32        45



In [62]:
# Crear el modelo
gbm = GradientBoostingClassifier()

# Definir el grid de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 1.0]
}

# Configurar la búsqueda de hiperparámetros
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Entrenar el modelo
grid_search.fit(X_train_fin, y_train_XG)

# Mejor modelo
best_gbm = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred = best_gbm.predict(X_test_fin)

# Evaluar el modelo
accuracy = accuracy_score(y_test_XG, y_pred)
print(f'Best parameters: {grid_search.best_params_}')
print(f'Accuracy: {accuracy}')

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


/home/julian/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/julian/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/julian/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/julian/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50, subsample=0.8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50, subsample=0.8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50, subsample=0.8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50, subsample=0.8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50, subsample=1.0; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50, subsample=0.8; total time=   0.2s


/home/julian/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/julian/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/julian/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50, subsample=1.0; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50, subsample=1.0; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50, subsample=1.0; total time=   0.2s


...

[CV] END learning_rate=0.2, max_depth=7, min_samples_leaf=4, min_samples_split=10, n_estimators=200, subsample=1.0; total time=   0.9s
[CV] END learning_rate=0.2, max_depth=7, min_samples_leaf=4, min_samples_split=10, n_estimators=200, subsample=0.8; total time=   3.4s


/home/julian/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/julian/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END learning_rate=0.2, max_depth=7, min_samples_leaf=4, min_samples_split=10, n_estimators=200, subsample=0.8; total time=   4.3s
[CV] END learning_rate=0.2, max_depth=7, min_samples_leaf=4, min_samples_split=10, n_estimators=200, subsample=1.0; total time=   0.9s


/home/julian/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END learning_rate=0.2, max_depth=7, min_samples_leaf=4, min_samples_split=10, n_estimators=200, subsample=1.0; total time=   3.6s
[CV] END learning_rate=0.2, max_depth=7, min_samples_leaf=4, min_samples_split=10, n_estimators=200, subsample=1.0; total time=   3.4s
[CV] END learning_rate=0.2, max_depth=7, min_samples_leaf=4, min_samples_split=10, n_estimators=200, subsample=1.0; total time=   2.9s
Best parameters: {'learning_rate': 0.1, 'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200, 'subsample': 1.0}
Accuracy: 0.35555555555555557


In [63]:
print(classification_report(y_test_XG, y_pred))

              precision    recall  f1-score   support

           0       0.31      0.73      0.43        15
           1       0.83      0.33      0.48        15
           2       0.00      0.00      0.00        15

    accuracy                           0.36        45
   macro avg       0.38      0.36      0.30        45
weighted avg       0.38      0.36      0.30        45

